# Project 2

# Purpose

This [November 2021 CNBC article](https://www.cnbc.com/2021/11/09/bitcoin-vs-gold-leading-gold-authorities-on-inflation-hedge-battle.html) on Bitcoin and gold as inflation and market risk hedges motivated this project.
I have two goals for this project:

1. To help you master data analysis
1. To help you evaluate articles in the popular media using your data analysis skills

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)
%precision 2
%config InlineBackend.figure_format = 'retina'

In [ ]:
import yfinance as yf
import pandas_datareader as pdr
import requests_cache
session = requests_cache.CachedSession()

In [ ]:
import scipy.optimize as sco
import seaborn as sns
import statsmodels.formula.api as smf

In [ ]:
# Download Bitcoin data
btc = (
    yf.download(tickers='BTC-USD', progress=False)
    .assign(Date=lambda x: x.index.tz_localize(None))
    .set_index('Date')
    .rename_axis(columns='Variable')
)

# Download Gold data
gold = (
    yf.download(tickers='GLD', progress=False)
    .assign(Date=lambda x: x.index.tz_localize(None))
    .set_index('Date')
    .rename_axis(columns='Variable')
)

# Download PCEPI data from FRED
pcepi = pdr.DataReader(
    'PCEPI',
    start='1900',
    data_source='fred'
)

# Download SP100 data
wiki = pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')
wiki_ticker=(wiki[2]['Symbol']).apply(lambda x: x.replace('.', '-')).tolist()
wiki_ticker
SP100 = (
    yf.download(tickers=wiki_ticker, progress=False)
    .assign(Date=lambda x: x.index.tz_localize(None))
    .set_index('Date')
    .rename_axis(columns=['Variable', 'Ticker'])
    
)

# Download French Fama data 
ff = (
    pdr.DataReader(
        name='F-F_Research_Data_Factors_daily',
        data_source='famafrench',
        start='1900',
        session=session
    )
    [0]
    .assign(Mkt = lambda x: x['Mkt-RF'] + x['RF'])
    .div(100)
)

def port_vol(x, r, ppy):
    return np.sqrt(ppy) * r.dot(x).std()

def port_mean(x, r, ppy):
    return ppy * r.dot(x).mean()

In [ ]:
btc

In [ ]:
ff

In [ ]:
gold

In [ ]:
pcepi

In [ ]:
SP100

# Tasks

## Task 1: Do Bitcoin and gold hedge inflation risk?

Use the typical finance definition of [hedge](https://www.investopedia.com/terms/h/hedge.asp):

> To hedge, in finance, is to take an offsetting position in an asset or investment that reduces the price risk of an existing position. A hedge is therefore a trade that is made with the purpose of reducing the risk of adverse price movements in another asset. Normally, a hedge consists of taking the opposite position in a related security or in a derivative security based on the asset to be hedged. 

Here are a few suggestions:

1. Measure Bitcoin's price with [BTC-USD](https://finance.yahoo.com/quote/BTC-USD?p=BTC-USD&.tsrc=fin-srch) and gold's price with [GLD](https://finance.yahoo.com/quote/GLD?p=GLD&.tsrc=fin-srch)
1. Throughout the project, assume Bitcoin and U.S. public equity markets have the same closing time
1. Measure the price level with [PCEPI](https://fred.stlouisfed.org/series/PCEPI/) from the Federal Reserve Database (FRED), which is downloadable with `pdr.DataReader()`
1. Measure inflation (i.e., the rate of change in the price level) as the percent change in PCEPI

In [ ]:
# Plot results
plt.figure(figsize=(8, 4))
plt.plot(inflation.index, inflation, label='Inflation Rate')
plt.xlabel('Year')
plt.ylabel('Price')
plt.legend()
plt.title('Inflation')
plt.show()

In [ ]:
# Calculate price and inflation rate
btc_price = btc['Adj Close'].resample('MS').last()
gold_price = gold['Adj Close'].resample('MS').last()
inflation = pcepi['PCEPI']

btc_inflation_corr = btc_price.corr(inflation)
gold_inflation_corr = gold_price.corr(inflation)

print('Correlation between Bitcoin and Inflation: {:.7f}'.format(btc_inflation_corr))
print('Correlation between Gold and Inflation: {:.7f}'.format(gold_inflation_corr))

fig, ax1 = plt.subplots(figsize=(8, 4))

# Slice the data from 2014 onwards
btc_price = btc_price.loc['2014':]
gold_price = gold_price.loc['2014':]
inflation = inflation.loc['2014':]

# plot Bitcoin prices on the first y-axis scale
color = 'tab:red'
ax1.set_xlabel('Date')
ax1.set_ylabel('Bitcoin Price', color=color)
ax1.plot(btc_price.index, btc_price, color=color)
ax1.tick_params(axis='y', labelcolor=color)

# create a second y-axis scale for Gold prices
ax2 = ax1.twinx()

# plot Gold prices on the second y-axis scale
color = 'tab:blue'
ax2.set_ylabel('Gold Price', color=color)
ax2.plot(gold_price.index, gold_price, color=color)
ax2.tick_params(axis='y', labelcolor=color)

ax3 = ax1.twinx()

# plot inflation on the third y-axis scale
color = 'tab:green'
ax3.spines['right'].set_position(('axes', 1.1))
ax3.set_ylabel('Inflation', color=color)
ax3.plot(inflation.index, inflation, color=color)
ax3.tick_params(axis='y', labelcolor=color)

# set title and show the plot
plt.title('Bitcoin and Gold Prices with inflation')
plt.show()

# Plot results
plt.figure(figsize=(8, 4))
plt.plot(btc_price.index, btc_price, label='Bitcoin Price')
plt.xlabel('Year')
plt.ylabel('Price')
plt.legend()
plt.title('Bitcoin Price')
plt.show()

# Plot results
plt.figure(figsize=(8, 4))
plt.plot(gold_price.index, gold_price, label='Gold Price')
plt.xlabel('Year')
plt.ylabel('Price')
plt.legend()
plt.title('Gold Price')
plt.show()

btc_monthlyret = btc['Adj Close'].resample('MS').last().pct_change().dropna()
gold_monthlyret = gold['Adj Close'].resample('MS').last().pct_change().dropna()

data_set = pd.merge(btc_monthlyret, gold_monthlyret, left_index=True, right_index=True)
data_set = pd.merge(data_set, inflation, left_index=True, right_index=True)
data_set.columns = ['BTC', 'Gold', 'Inflation']

# Define the formula for the regression
f = 'BTC ~ Inflation'
f1 = 'Gold ~ Inflation'

# Estimate beta coefficients for Bitcoin using OLS & print regression results for Bitcoin
btc_model = smf.ols(formula=f, data=data_set).fit()
btc_model.summary()

# Estimate beta coefficients for Gold using OLS & print regression results for Gold
gold_model = smf.ols(formula=f1, data=data_set).fit()
gold_model.summary()

Task 1 


Based on the data above, the conclusion is that both Bitcoin and gold do not appear to effectively hedge against inflation risk.

The expected return for Bitcoin is 79.11% and for gold it is 70.58%, indicating that both assets have shown positive returns historically. However, the correlation between Bitcoin and inflation is -0.01, and the correlation between gold and inflation is -0.04, which suggests a weak or negligible relationship between the two assets and inflation. But, correlation does not necessarily imply causation, and hedge against inflation risk is a complex concept that depends on various factors beyond just correlation. It is important to note that correlation alone does not provide conclusive evidence of hedging against inflation risk.

Furthermore, the volatility (or risk) of Bitcoin is 64.99% and for gold it is 15.56%, indicating that Bitcoin is significantly more volatile compared to gold. The plot of Bitcoin and gold returns compared to the inflation rate also does not show a clear pattern of these assets effectively hedging against inflation risk, as their returns do not consistently move in opposite directions to inflation.

Therefore, based on the data and analysis, it can be concluded that Bitcoin and gold may not be reliable hedges against inflation risk. Investors should consider other assets or strategies for effectively hedging against inflation.

To better assess whether Bitcoin and gold hedge inflation risk, additional analysis and considerations are needed, such as examining the historical price trends, volatility, and performance during inflationary periods, as well as considering other factors such as market dynamics, economic conditions, and investment objectives. It may also be prudent to consult with a financial professional or conduct a comprehensive analysis using additional financial and economic data to make informed investment decisions.

Task 1 

Based on the data above, the conclusion is that both Bitcoin and gold probably could effectively hedge against inflation risk. Bitcoin and Gold, its indicating that both assets have shown positive returns historically. However, the correlation between Bitcoin and inflation is 0.7137893, and the correlation between gold and inflation is 0.7924957, which suggests a strong or possitive relationship between the two assets and inflation. But, correlation does not necessarily imply causation, and hedge against inflation risk is a complex concept that depends on various factors beyond just correlation. It is important to note that correlation alone does not provide conclusive evidence of hedging against inflation risk. 

The correlation values you provided indicate the strength and direction of the relationship between Bitcoin, gold, and inflation. A correlation coefficient ranges from -1 to 1, where values closer to -1 or 1 indicate a stronger correlation, while values closer to 0 suggest a weaker or no correlation. The positive correlation values you mentioned suggest a positive relationship between both Bitcoin and gold with inflation.

Correlation between Bitcoin and Inflation: 0.7137893
A correlation of 0.7137893 indicates a moderately strong positive relationship between Bitcoin and inflation. This means that historically, Bitcoin prices have tended to move in the same direction as inflation, albeit not perfectly. When inflation increases, Bitcoin prices have, on average, shown a tendency to rise as well. However, it's important to note that correlation does not imply causation. Other factors may also influence Bitcoin prices, such as market sentiment, adoption, regulatory developments, and technological advancements.

Correlation between Gold and Inflation: 0.7924957
A correlation of 0.7924957 indicates a relatively strong positive relationship between gold and inflation. This suggests that historically, gold prices have tended to rise along with inflation. When inflation increases, the value of gold has, on average, shown a propensity to increase as well. Gold has a long-standing reputation as a hedge against inflation, and this correlation value supports that notion. However, similar to Bitcoin, gold prices can also be influenced by other factors such as geopolitical events, market demand, and central bank policies.

It's worth noting that correlation alone does not imply that Bitcoin and gold are foolproof inflation hedges. Correlation measures the statistical relationship between variables but does not account for other factors that may impact their value. Moreover, correlation can change over time due to shifting market dynamics and changing economic conditions.

When considering Bitcoin and gold as potential inflation hedges, it's crucial to assess other factors such as market volatility, liquidity, investor sentiment, and individual risk tolerance. Additionally, diversification across various asset classes can help mitigate risk and enhance the overall resilience of an investment portfolio. Consulting with a financial advisor can provide personalized insights and guidance based on your specific financial goals and circumstances.

Ordinary Least Squares (OLS) regression analysis, which is a statistical technique used to estimate the relationship between a dependent variable and one or more independent variables. In this case, the dependent variable is "Gold," and the independent variable is "Inflation."

With R-squared: 0.000
The R-squared value measures the proportion of the variance in the dependent variable that is explained by the independent variable(s). In this case, the R-squared value of 0.000 suggests that the independent variable "Inflation" does not explain a significant amount of the variation in the dependent variable "Gold." In other words, the inflation variable does not have a strong linear relationship with gold prices in the given model.

OLS regression analysis focuses on the relationship between the dependent variable "BTC" (Bitcoin) and the independent variable "Inflation." Let's interpret the important components of the regression output:

With R-squared: 0.017
The R-squared value measures the proportion of the variance in the dependent variable that is explained by the independent variable(s). In this case, the R-squared value of 0.017 suggests that the independent variable "Inflation" explains a very small portion of the variation in Bitcoin prices. This means that inflation alone does not account for a significant amount of the changes observed in Bitcoin prices in the given model.

Similar to gold, Bitcoin isn't tied to a particular currency or economy. It also isn't controlled by a small group of companies or stakeholders. Rather, it is an international asset class that reflects global demand. In times of high U.S. inflation, investors must take on more risk to offset the decline in existing asset values. For example, a 3% dividend yield may normally supplement income in retirement. But if inflation is 6%, it simply isn't good enough. Bitcoin could be one of the better options outside of equities because it sidesteps many of the political and economic risks associated with the U.S. stock market. In this vein, Bitcoin and other cryptocurrencies are one of the most practical and simple ways for an American to diversify away from purely American revenue, income, and assets.

Lessons about gold’s track record as an investing hedge may be learned by looking back at the 1970s, when the U.S. experienced its last bout of high inflation.Oil price shocks and energy shortages drove average annual U.S. inflation up to around 8.8% from 1973 to 1979. During those six years, gold won over many investors as a top inflation hedge, since the yellow metal generated an impressive 35% annual return.

## Task 2: Do Bitcoin and gold hedge market risk?

Here are a few suggestions:

1. Estimate capital asset pricing model (CAPM) regressions for Bitcoin and gold
1. Use the daily factor data from Ken French

In [ ]:
# Calculate returns and inflation rate
btc_returns = btc['Adj Close'].pct_change().dropna()
gold_returns = gold['Adj Close'].pct_change().dropna()

In [ ]:
# Merge data into a single data frame
data = pd.merge(btc_returns, gold_returns, left_index=True, right_index=True)
data = pd.merge(data, ff['RF'], left_index=True, right_index=True)
data = pd.merge(data, ff['Mkt-RF'], left_index=True, right_index=True)
data.columns = ['BTC', 'Gold', 'RF', 'RP']

In [ ]:
ff_factors = ff[['Mkt-RF', 'RF']]

In [ ]:
# Define the formula for the regression
formula = 'BTC - R ~ RP - RF'
formula_1 = 'Gold - RF ~ RP - RF'

In [ ]:
# Estimate beta coefficients for Bitcoin using OLS & print regression results for Bitcoin
btc_model = smf.ols(formula=formula, data=data).fit()
beta_btc = btc_model.params['RP']
btc_model.summary()

In [ ]:
# Estimate beta coefficients for Gold using OLS & print regression results for Gold
gold_model = smf.ols(formula=formula_1, data=data).fit()
beta_gold = gold_model.params['RP']
gold_model.summary()

In [ ]:
rf = ff_factors['RF'].mean()
market_return = ff_factors['Mkt-RF'].mean()

# Estimate expected return for Bitcoin using CAPM equation
expected_return_btc = rf + beta_btc * (market_return - rf)
print('Expected return for Bitcoin: {:.7%}'.format(expected_return_btc))

# Estimate expected return for gold using CAPM equation
expected_return_gold = rf + beta_gold * (market_return - rf)
print('Expected return for Gold: {:.7%}'.format(expected_return_gold))

In [ ]:
# Create a scatter plot of BTC returns vs. market returns
plt.scatter(data['RM'], data['BTC'])
plt.xlabel('Market Returns')
plt.ylabel('Bitcoin Returns')
plt.title('Scatter Plot of Bitcoin Returns vs. Market Returns')

## Task 3: Plot the mean-variance efficient frontier of Standard & Poor's 100 Index (SP100) stocks, with and without Bitcoin and gold

Here are a few suggestions:

1. You can learn about the SP100 stocks [here](https://en.wikipedia.org/wiki/S%26P_100)
1. Only consider days with complete data for Bitcoin and gold
1. Drop any stocks with shorter return histories than Bitcoin and gold
1. Assume long-only portfolios

## Task 4: Find the maximum Sharpe Ratio portfolio of SP100 stocks, with and without Bitcoin and gold

Follow the data requirements of task 3.

## Task 5: Every full calendar year, compare the $\frac{1}{n}$ portfolio with the out-of-sample performance of the previous maximum Sharpe Ratio portfolio

Follow the data requirements of task 3.
Estimate the previous maximum Sharpe Ratio portfolio using data from the previous two years.
Consider, at least, the Sharpe Ratios of each portfolio, but other performance measures may help you tell a more complete story.

## Task 6: What do you conclude about Bitcoin and gold as inflation and market risk hedges?

In [ ]:
#

What are your overall conclusions and limitations of your analysis?
What do the data suggest about the article that motivated this project?
Please see the link at the top of this notebook.

# Criteria

1. ***Discuss and explain your findings for all 6 tasks, and be specific!***
1. ***Your goal is to convince me of your calculations and conclusions***
1. All tasks are worth 16.67 points each
1. Your report should not exceed 25 pages
1. Here are more tips
    1. Each task includes suggestions
    1. I suggest you include plots and calculations for all but the last task
    1. Remove unnecessary code, outputs, and print statements
    1. Write functions for plots and calculations that you use more than once
    1. I will not penalize code style, but I will penalize submissions that are difficult to follow or do not follow these instructions
1. How to submit your project
    1. Restart your kernel, run all cells, and save your notebook
    1. Export your notebook to PDF (`File > Save And Export Notebook As ... > PDF` in JupyterLab)
        1. If this export does not work, you can either (1) Install MiKTeX on your laptop with default settings or (2) use DataCamp Workspace to export your notebook to PDF
        1. You do not need to re-run your notebook to export it because notebooks store output cells
    1. Upload your notebook and PDF to Canvas
    1. Upload your PDF only to Gradescope and tag your tasks and teammates
    1. Gradescope helps me give better feedback more quickly, but it is not reliable for sharing and storing your submission files